## Tensorflow Dataloader Profiling
Swapping to TF I/O.

This notebook moves image loading into tf.io.read_file which is threadsafe. Changes:
- Move from CV2 to TF.image_read
- Replace OpenCV with TensorFlow's image processing functions to leverage GPU acceleration
- Increase shuffle buffer size
- Reduce the prefetch size to 3 instead of AutoTUNE

In [0]:
# Some quality of life fixes for logging
import os
import urllib3
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
urllib3.connection.HTTPConnection.default_pool_maxsize = 50
urllib3.connection.HTTPSConnection.default_pool_maxsize = 50

In [0]:
import tensorflow as tf
import json
import numpy as np
from pathlib import Path

Same function but now we use Tensorflow IO

In [0]:
def create_dataset(
    image_paths: list, 
    batch_size: int = 32, 
    img_shape=(224, 224), 
    max_objects=30,
    use_random=True):
    """Creates a dataset from a list of image paths with optimized performance using TensorFlow IO"""
    
    # Create dataset from tensor slices for better performance
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    
    if use_random:
        # Use a larger buffer for better randomization, but not too large to avoid memory issues
        buffer_size = min(len(image_paths), 10000)
        dataset = dataset.shuffle(
            buffer_size=buffer_size, 
            reshuffle_each_iteration=True
            )
    
    # Cache file paths before expensive operations
    dataset = dataset.cache()
    
    def _load_image(image_path):
        """
        Load image using TensorFlow operations
        Do TensorFlow operations seperately
        """
        img_data = tf.io.read_file(image_path)
        img = tf.io.decode_png(img_data, channels=3)
        img = tf.image.resize(img, img_shape)
        img = tf.cast(img, tf.float32) / 255.0
        return img
    
    def _load_annotation(image_path_tensor):
        """
        Python function to load annotations
        JSON parsing is not available in TF ops
        Needs a py_function
        """
        image_path = image_path_tensor.numpy().decode('utf-8')
        annotation_path = image_path.replace('.png', '.json')
        with open(annotation_path, 'r') as f:
            ann = json.load(f)
        
        bboxes = np.array(ann['objects']['bbox'], dtype=np.float32)
        categories = np.array(ann['objects']['category'], dtype=np.int32)
        
        # Pad bboxes and categories
        padded_bboxes = np.zeros((max_objects, 4), dtype=np.float32)
        padded_categories = np.full((max_objects), -1, dtype=np.int32)
        
        num_objects = min(len(bboxes), max_objects)
        padded_bboxes[:num_objects] = bboxes[:num_objects]
        padded_categories[:num_objects] = categories[:num_objects]
        
        return padded_bboxes, padded_categories
    
    def _process_path(image_path):
        """Process a single image path"""
        image = _load_image(image_path)

        bboxes, categories = tf.py_function(
            _load_annotation,
            [image_path],
            [tf.float32, tf.int32]
        )
        
        image = tf.ensure_shape(image, (img_shape[0], img_shape[1], 3))
        bboxes_out = tf.ensure_shape(bboxes, (max_objects, 4))
        categories_out = tf.ensure_shape(categories, (max_objects,))

        return {'images': tf.cast(image, tf.float32, name='images')}, {
            'bboxes': tf.cast(bboxes_out, tf.float32, name='bboxes'),
            'classes': tf.cast(categories_out, tf.int32, name='classes')
        }
    
    dataset = dataset.map(_process_path, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.cache()
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

In [0]:
from pathlib import Path
image_paths = [str(x) for x in Path('/Volumes/shm/default/cppe5/').glob('*.png')]
dataset = create_dataset(image_paths[0:100])

In [0]:
for batch in dataset.take(1):
    images = batch[0]['images']
    bboxes = batch[1]['bboxes']
    categories = batch[1]['classes']
    print(f"Input shape: {images.shape}")
    print(f"Bounding boxes shape: {bboxes.shape}")
    print(f"Categories shape: {categories.shape}")

In [0]:
from model import build_object_detection_model
model = build_object_detection_model()

Same steps per epoch, but now we are using the Tensorflow I/O dataset. The libpng warning dissapear. This seems to reduce data loading on the GPU significantly

In [0]:
steps_per_epoch = len(image_paths) // 32
model.fit(dataset, epochs=1, steps_per_epoch=steps_per_epoch)